<a href="https://colab.research.google.com/github/makiwarner/NLP_2025_Assignment1/blob/main/NLP_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import string
from collections import defaultdict
import random


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path =  "/content/drive/MyDrive/shakespeare.txt"


**Task 1: Data Preparation**

Below, I am opening the Shakespearean text file that I saved in my drive in read mode.

Then, I combine all lines, covert all letters to lowercase, and remove the punctuation. I do this with the text.translate() function which maps all the punctuation characters to '' - None.

From there, I split the text into tokens (words) based on the white spaces.

In [4]:
def load_and_preprocess_text(path):
    with open(path, 'r') as file:
        lines = file.readlines()

    text = ' '.join(lines)  # Combine all lines into a single string

    text = text.lower()

    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = text.split()

    return tokens

The create_bigrams() function is quite straightfoward: we loop through all the tokens and form a pair of each word + the following word.

In [5]:
def create_bigrams(tokens):
    bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1)]
    return bigrams

The goal of the fill_bigram_to_next_token_counts() function is to give us an idea of the possibility of words (tokens) that follow each bigram. Thus, it is necessary that both the bigrams and tokens are taken as inputs.

From there, I established the structure of the returned output (from_bigram_to_next_token_counts) to be a dictionary. (defaultdict will apply 0 if the next word is not present). The dictionary will take a bigram as a key and another dictionary of the next token and its count as the value.



In [6]:
def fill_bigram_to_next_token_counts(bigrams, tokens):
    from_bigram_to_next_token_counts = defaultdict(lambda: defaultdict(int))

    for i in range(len(bigrams)):
        bigram = bigrams[i]
        if i + 2 < len(tokens):  #ensure there's a next token after the bigram
            next_token = tokens[i + 2]
            from_bigram_to_next_token_counts[bigram][next_token] += 1

    return from_bigram_to_next_token_counts

**Task 2: Probability Distribution**

Calculate probabilities for tokens that follow each bigram.

This function uses the calculate_bigram_probabilities() function to transform the dictionary from next_possible_word : count, to contain the probability of the next word instead.

In [7]:
def calculate_bigram_probabilities(from_bigram_to_next_token_counts):
    from_bigram_to_next_token_probs = {}

    for bigram, next_token_counts in from_bigram_to_next_token_counts.items():
        total_count = sum(next_token_counts.values())
        from_bigram_to_next_token_probs[bigram] = {
            token: count / total_count for token, count in next_token_counts.items()
        }

    return from_bigram_to_next_token_probs

**Task 3: Sampling Next Token**

Sample the next token based on the probability distribution.

The goal here is to predict the next word that follows a given bigram using a weighted random sampling method. It takes into account both the actually probability of the word appearing, and randomness. Interestingly, this is more reflective of the real world than deterministically choosing the word with the highest probability.

In [8]:
def sample_next_token(bigram, bigram_to_next_token_probs):
    if bigram not in bigram_to_next_token_probs:
        return None  #return None if bigram is not in the dictionary

    next_tokens = list(bigram_to_next_token_probs[bigram].keys())
    probabilities = list(bigram_to_next_token_probs[bigram].values())

    #random.choices for weighted sampling:
    sampled_token = random.choices(next_tokens, weights=probabilities, k=1)[0]
    return sampled_token

**Task 4: Generating Text**

Generate text starting with the given bigram and sampling the next token iteratively.

We rely on the above sample_next_token() function to generate our text.

We start by applying sample_next_token() to the bigram in order to generate the 3rd word, and we append these words to the generated_text array. We continually repeat this process until we go through all bigrams and generate the next word.

In [9]:
def generate_text_from_bigram(start_bigram, num_words, bigram_to_next_token_probs):
    current_bigram = start_bigram
    generated_text = [current_bigram[0], current_bigram[1]]  # Start with the initial bigram

    for _ in range(num_words - 2):  # Already have two words from the bigram
        next_token = sample_next_token(current_bigram, bigram_to_next_token_probs)
        if not next_token:
            break  # Stop if no next token can be sampled

        generated_text.append(next_token)
        # Update the bigram to the most recent pair of words
        current_bigram = (current_bigram[1], next_token)

    return ' '.join(generated_text)

**Test cases** for each function above - to print the output and verify the desired output

In [10]:
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/shakespeare.txt"

    tokens = load_and_preprocess_text(file_path)

    bigrams = create_bigrams(tokens)

    bigram_to_next_token_counts = fill_bigram_to_next_token_counts(bigrams, tokens)

    bigram_to_next_token_probs = calculate_bigram_probabilities(bigram_to_next_token_counts)

    example_bigram = ('to', 'be')
    sampled_token = sample_next_token(example_bigram, bigram_to_next_token_probs)
    print(f"Sampled next token for {example_bigram}: {sampled_token}") #verification of output

    start_bigram = ('to', 'be')
    num_words = 50
    generated_text = generate_text_from_bigram(start_bigram, num_words, bigram_to_next_token_probs)
    print(f"Generated text: {generated_text}") #verification of output

    # Verification of outputs:
    example_bigram = ('to', 'be')
    if example_bigram in bigram_to_next_token_probs:
        print(f"Next token counts for {example_bigram}: {bigram_to_next_token_counts[example_bigram]}")
        print(f"Next token probabilities for {example_bigram}: {bigram_to_next_token_probs[example_bigram]}")
    else:
        print(f"The bigram {example_bigram} does not exist in the text.")
    if sampled_token:
        print(f"Sampled next token for {example_bigram}: {sampled_token}")
    else:
        print(f"The bigram {example_bigram} does not exist in the probabilities dictionary.")


Generated text: to be crossed prison my heart which i by this thou dost common grow that thou consumst thy self art so possessed with murdrous hate that gainst thy self to breed another thee or if they sing tis with so dull a cheer that leaves look pale dreading the winters
Next token counts for ('to', 'be'): defaultdict(<class 'int'>, {'new': 1, 'die': 1, 'gone': 1, 'those': 1, 'deaths': 1, 'won': 1, 'assailed': 1, 'remembered': 1, 'with': 1, 'your': 1, 'praised': 1, 'then': 1, 'diseased': 1, 'vile': 1, 'receives': 1, 'so': 1, 'sure': 1, 'crossed': 1, 'my': 1, 'if': 1, 'invited': 1, 'only': 1, 'a': 1, 'beloved': 1, 'to': 1})
Next token probabilities for ('to', 'be'): {'new': 0.04, 'die': 0.04, 'gone': 0.04, 'those': 0.04, 'deaths': 0.04, 'won': 0.04, 'assailed': 0.04, 'remembered': 0.04, 'with': 0.04, 'your': 0.04, 'praised': 0.04, 'then': 0.04, 'diseased': 0.04, 'vile': 0.04, 'receives': 0.04, 'so': 0.04, 'sure': 0.04, 'crossed': 0.04, 'my': 0.04, 'if': 0.04, 'invited': 0.04, 'only':

**Task 5**

 Change the bigram specific functions to n grams, and compare the results to the bigram performance.

In [11]:
def create_ngrams(tokens, n):
    ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    return ngrams

def fill_ngram_to_next_token_counts(ngrams, tokens, n): #create a dictionary where keys are n-grams and values are dictionaries of following token counts.
    ngram_to_next_token_counts = defaultdict(lambda: defaultdict(int))

    for i in range(len(ngrams)):
        ngram = ngrams[i]
        if i + n < len(tokens):  # Ensure there's a next token after the n-gram
            next_token = tokens[i + n]
            ngram_to_next_token_counts[ngram][next_token] += 1

    return ngram_to_next_token_counts

def calculate_ngram_probabilities(ngram_to_next_token_counts):
    ngram_to_next_token_probs = {}

    for ngram, next_token_counts in ngram_to_next_token_counts.items():
        total_count = sum(next_token_counts.values())
        ngram_to_next_token_probs[ngram] = {
            token: count / total_count for token, count in next_token_counts.items()
        }

    return ngram_to_next_token_probs

def sample_next_token(ngram, ngram_to_next_token_probs):
    if ngram not in ngram_to_next_token_probs:
        return None  # if the n-gram is not in the dictionary

    next_tokens = list(ngram_to_next_token_probs[ngram].keys())
    probabilities = list(ngram_to_next_token_probs[ngram].values())

    sampled_token = random.choices(next_tokens, weights=probabilities, k=1)[0]
    return sampled_token

def generate_text_from_ngram(start_ngram, num_words, ngram_to_next_token_probs):
    current_ngram = start_ngram
    generated_text = list(current_ngram)  #start with the initial n-gram

    for _ in range(num_words - len(start_ngram)):
        next_token = sample_next_token(current_ngram, ngram_to_next_token_probs)
        if not next_token:
            break

        generated_text.append(next_token)
        # Update the n-gram to include the most recent tokens
        current_ngram = tuple(generated_text[-len(current_ngram):])

    return ' '.join(generated_text)



Using 2-grams:
Generated text: the sonnets by william shakespeare from fairest creatures we desire increase that thereby beautys rose might never die but if the while i think good thoughts whilst other write good words and in my thought whose love to you as he shows now my greatest grief thou best of dearest

Using 3-grams:
Generated text: the sonnets by william shakespeare from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a

Using 4-grams:
Generated text: the sonnets by william shakespeare from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a


**Test Cases** for task 5

In [ ]:
if __name__ == "__main__":
    file_path = "/content/drive/MyDrive/shakespeare.txt"

    tokens = load_and_preprocess_text(file_path)

    n_values = [2, 3, 4]  # Bigrams, Trigrams, Quadgrams

    for n in n_values:
        print(f"\nUsing {n}-grams:")

        ngrams = create_ngrams(tokens, n)

        ngram_to_next_token_counts = fill_ngram_to_next_token_counts(ngrams, tokens, n)

        ngram_to_next_token_probs = calculate_ngram_probabilities(ngram_to_next_token_counts)

        start_ngram = tuple(tokens[:n])
        num_words = 50
        generated_text = generate_text_from_ngram(start_ngram, num_words, ngram_to_next_token_probs)

        print(f"Generated text: {generated_text}")

**Comparison between the bigram, trigram, and quadgram:**

I was surprised to see that all 3 models predicted the same first 18 words! Especially given that we are using a random weighted sampling method, I would have thought the stochastic element would make the word predictions different.

Once the words began to deviate, it is clear that the 4-gram model is best given the words are more related to eachother.  However, it is still not very logicial, as it starts by talking about rose, then an heir, then eyes with flames and fuel.